In [ ]:
import os
import torch
from tqdm import tqdm
import glob
import pathlib
import h5py
import numpy as np
from modules.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from modules.modeling import CLIP4Clip
from modules.tokenization_clip import SimpleTokenizer as ClipTokenizer
from PIL import Image
from torchvision import transforms
import json
from open_clip import get_tokenizer
import networkx as nx

In [ ]:
class args:
    msvd = False # or msvd = False for MSR-VTT
    slice_framepos = 2
    dset = "../"
    max_frames = 20
    eval_frame_order = 0
    output_dir = 'pretrained'
    cache_dir = ''
    features_path = '..'
    msrvtt_csv = 'msrvtt.csv'
    max_words =32
    feature_framerate = 1
    cross_model = "cross-base"
    local_rank = 0
    pretrained_clip_name = "ViT-B/16"

In [ ]:
model_file = "ckpts/ckpt_msrvtt_retrieval_looseType/pytorch_model.bin.4"
model_state_dict = torch.load(model_file, map_location='cpu')
cache_dir = args.cache_dir if args.cache_dir else os.path.join(str(PYTORCH_PRETRAINED_BERT_CACHE), 'distributed')
model = CLIP4Clip.from_pretrained(args.cross_model, cache_dir=cache_dir, state_dict=model_state_dict, task_config=args)
tokenizer = get_tokenizer('ViT-B-32')
clip = model.clip.cuda()

In [ ]:
res = {} 
# res[video_id] = {'entities': [], 'relations': []}

In [ ]:
with h5py.File("clip_embeddings.hdf5", "w") as fs:
    for key in tqdm(res.keys()):
        ent_tokens = tokenizer(res[key]['entities'])
        ent_embedding = clip.encode_text(ent_tokens.cuda())

        rel_tokens = tokenizer(res[key]['relations'])
        rel_embedding = clip.encode_text(rel_tokens.cuda())
        embeddings= torch.cat([ent_embedding, rel_embedding]).cpu().detach().numpy()
        fs.create_dataset(key, data=embeddings)